In [ ]:
# VibeCurator Offline Evaluation Pipeline
# Stage 1 / 1.5 / 2 공통 오프라인 평가 노트북

# ============================================================================
# [Cell 1] 기본 설정 및 공통 import
# ============================================================================

import os
import json
import random
from typing import List, Dict, Callable, Tuple
import numpy as np
import pandas as pd

# pandas 표시 설정
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 120)

# 랜덤 시드 고정
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# 경로 상수 정의 (recommend_model 폴더 기준)
DATA_DIR = "../melon-dataset-excepttar"  # recommend_model 폴더 기준
TRAIN_JSON = os.path.join(DATA_DIR, "train.json")
SONG_META_JSON = os.path.join(DATA_DIR, "song_meta.json")

MODEL_DIR = "../models"  # recommend_model 폴더 기준
STAGE1_MODEL_PATH = os.path.join(MODEL_DIR, "v2_item2vec.model")

EVAL_OUTPUT_DIR = "../eval"  # recommend_model 폴더 기준
os.makedirs(EVAL_OUTPUT_DIR, exist_ok=True)

print("✅ 기본 설정 완료")
print(f"  - Random Seed: {RANDOM_SEED}")
print(f"  - Eval Output: {EVAL_OUTPUT_DIR}")


In [ ]:
# ============================================================================
# [Cell 2] song_meta 로드 및 메타데이터 준비
# ============================================================================

print("\n" + "="*60)
print("📂 song_meta.json 로드 중...")
print("="*60)

with open(SONG_META_JSON, 'r', encoding='utf-8') as f:
    song_meta_raw = json.load(f)

# DataFrame 생성
song_meta_list = []
for item in song_meta_raw:
    song_id = item.get('id')
    song_name = item.get('song_name', 'UNKNOWN')
    
    # artist_name_basket → artist_str
    artist_basket = item.get('artist_name_basket', [])
    artist_str = ", ".join(artist_basket) if artist_basket else "UNKNOWN"
    
    # artist_id_basket → artist_key (첫 번째 ID)
    artist_id_basket = item.get('artist_id_basket', [])
    artist_key = str(artist_id_basket[0]) if artist_id_basket else "UNKNOWN"
    
    # song_gn_gnr_basket → main_genre (첫 번째 장르)
    genre_basket = item.get('song_gn_gnr_basket', [])
    main_genre = genre_basket[0] if genre_basket else "GN0000"
    
    # issue_date → issue_year
    issue_date = item.get('issue_date', '')
    try:
        issue_year = int(str(issue_date)[:4]) if issue_date else 0
    except:
        issue_year = 0
    
    song_meta_list.append({
        'song_id': song_id,
        'song_name': song_name,
        'artist_str': artist_str,
        'artist_key': artist_key,
        'main_genre': main_genre,
        'issue_year': issue_year
    })

song_meta_df = pd.DataFrame(song_meta_list)
song_meta_df.set_index('song_id', inplace=True)

print(f"✅ 메타데이터 로드 완료: {len(song_meta_df):,}곡")
print(song_meta_df.head())

# lookup_song_meta 헬퍼 함수
def lookup_song_meta(song_id: int) -> dict:
    """곡 메타데이터를 dict로 반환"""
    if song_id not in song_meta_df.index:
        return {
            'song_name': 'UNKNOWN',
            'artist_str': 'UNKNOWN',
            'artist_key': 'UNKNOWN',
            'main_genre': 'GN0000',
            'issue_year': 0
        }
    row = song_meta_df.loc[song_id]
    return row.to_dict()

In [ ]:
# ============================================================================
# [Cell 3] Stage 1 모델 로드 및 IN_CATALOG_SONGS 정의
# ============================================================================

print("\n" + "="*60)
print("🔄 Stage 1 모델 로드 중...")
print("="*60)

from gensim.models import Word2Vec

stage1_model = Word2Vec.load(STAGE1_MODEL_PATH)
IN_CATALOG_SONGS = {int(k) for k in stage1_model.wv.key_to_index.keys()}

print(f"✅ Stage 1 모델 로드 완료")
print(f"  - Vocab 크기: {len(IN_CATALOG_SONGS):,}곡")
print(f"  - Vector 차원: {stage1_model.wv.vector_size}")


In [ ]:
# ============================================================================
# [Cell 4] 플레이리스트 로드 및 in-catalog 필터링
# ============================================================================

print("\n" + "="*60)
print("📋 플레이리스트 로드 및 필터링 중...")
print("="*60)

with open(TRAIN_JSON, 'r', encoding='utf-8') as f:
    playlists = json.load(f)

MIN_LEN = 5  # 최소 플레이리스트 길이

ply_list = []
for ply in playlists:
    ply_id = ply.get('id', len(ply_list))
    songs = ply.get('songs', [])
    
    # IN_CATALOG_SONGS에 있는 곡만 필터링
    songs_in_vocab = [s for s in songs if s in IN_CATALOG_SONGS]
    
    # 최소 길이 체크
    if len(songs_in_vocab) >= MIN_LEN:
        ply_list.append({
            'playlist_id': ply_id,
            'song_ids': songs_in_vocab
        })

ply_df = pd.DataFrame(ply_list)

print(f"✅ 플레이리스트 필터링 완료")
print(f"  - 원본 플레이리스트: {len(playlists):,}개")
print(f"  - 필터 후 플레이리스트: {len(ply_df):,}개")
print(f"  - 평균 곡 수: {ply_df['song_ids'].apply(len).mean():.1f}곡")

In [ ]:
# ============================================================================
# [Cell 5] Train / Val / Test split 및 evaluation cases 생성
# ============================================================================

print("\n" + "="*60)
print("✂️ Train / Val / Test split 수행 중...")
print("="*60)

# 플레이리스트 인덱스 섞기
ply_indices = list(range(len(ply_df)))
random.shuffle(ply_indices)

# 80 / 10 / 10 split
n_total = len(ply_indices)
n_train = int(n_total * 0.8)
n_val = int(n_total * 0.1)

train_indices = ply_indices[:n_train]
val_indices = ply_indices[n_train:n_train+n_val]
test_indices = ply_indices[n_train+n_val:]

ply_train_df = ply_df.iloc[train_indices].reset_index(drop=True)
ply_val_df = ply_df.iloc[val_indices].reset_index(drop=True)
ply_test_df = ply_df.iloc[test_indices].reset_index(drop=True)

print(f"✅ Split 완료")
print(f"  - Train: {len(ply_train_df):,}개 ({len(ply_train_df)/n_total*100:.1f}%)")
print(f"  - Val:   {len(ply_val_df):,}개 ({len(ply_val_df)/n_total*100:.1f}%)")
print(f"  - Test:  {len(ply_test_df):,}개 ({len(ply_test_df)/n_total*100:.1f}%)")

# Evaluation cases 생성 (test용)
eval_cases = []
for idx, row in ply_test_df.iterrows():
    song_ids = row['song_ids']
    if len(song_ids) < 2:
        continue
    
    seed_song_id = song_ids[0]
    target_song_ids = song_ids[1:]
    
    eval_cases.append({
        'playlist_id': row['playlist_id'],
        'seed_song_id': seed_song_id,
        'target_song_ids': target_song_ids
    })

eval_cases_df = pd.DataFrame(eval_cases)

# 저장
eval_cases_path = os.path.join(EVAL_OUTPUT_DIR, "eval_cases.pkl")
eval_cases_df.to_pickle(eval_cases_path)

print(f"\n✅ Evaluation cases 생성 완료")
print(f"  - 생성된 cases: {len(eval_cases_df):,}개")
print(f"  - 저장 경로: {eval_cases_path}")
print(f"\n샘플 5개:")
print(eval_cases_df.head())

In [ ]:
# ============================================================================
# [Cell 6] 평가 지표 함수 정의
# ============================================================================

def recall_at_k(recommended: List[int], targets: List[int], k: int) -> float:
    """Recall@K 계산"""
    if not targets:
        return 0.0
    
    recommended_k = set(recommended[:k])
    targets_set = set(targets)
    
    hits = len(recommended_k & targets_set)
    return hits / len(targets_set)


def ndcg_at_k(recommended: List[int], targets: List[int], k: int) -> float:
    """nDCG@K 계산 (binary relevance)"""
    if not targets:
        return 0.0
    
    targets_set = set(targets)
    
    # DCG 계산
    dcg = 0.0
    for i, song_id in enumerate(recommended[:k], 1):
        rel = 1.0 if song_id in targets_set else 0.0
        dcg += rel / np.log2(i + 1)
    
    # IDCG 계산 (이상적인 순서: 모든 정답이 위에)
    ideal_length = min(k, len(targets))
    idcg = sum(1.0 / np.log2(i + 1) for i in range(1, ideal_length + 1))
    
    if idcg == 0:
        return 0.0
    
    return dcg / idcg


def unique_artists_and_genres_at_k(recommended: List[int]) -> Tuple[int, int]:
    """TopK에서 유니크한 아티스트 수와 장르 수 반환"""
    artists = []
    genres = []
    
    for song_id in recommended:
        meta = lookup_song_meta(song_id)
        artists.append(meta['artist_str'])
        genres.append(meta['main_genre'])
    
    return len(set(artists)), len(set(genres))


def max_artist_share_at_k(recommended: List[int]) -> float:
    """가장 많이 나온 아티스트의 점유율"""
    if not recommended:
        return 0.0
    
    artist_counts = {}
    for song_id in recommended:
        meta = lookup_song_meta(song_id)
        artist = meta['artist_str']
        artist_counts[artist] = artist_counts.get(artist, 0) + 1
    
    max_count = max(artist_counts.values())
    return max_count / len(recommended)


def seed_genre_ratio_at_k(seed_song_id: int, recommended: List[int]) -> float:
    """Seed 곡과 동일한 장르 비율"""
    if not recommended:
        return 0.0
    
    seed_meta = lookup_song_meta(seed_song_id)
    seed_genre = seed_meta['main_genre']
    
    same_genre_count = 0
    for song_id in recommended:
        meta = lookup_song_meta(song_id)
        if meta['main_genre'] == seed_genre:
            same_genre_count += 1
    
    return same_genre_count / len(recommended)


print("✅ 평가 지표 함수 정의 완료")


In [ ]:
# ============================================================================
# [Cell 7] Stage별 추천 함수 래퍼 정의
# ============================================================================

# TODO: 실제 구현에 맞게 아래 함수들을 수정하세요
# 현재는 Stage 1만 구현되어 있습니다

def recommend_stage1(seed_song_id: int, topk: int) -> List[int]:
    """
    Stage 1: item2vec 순수 CF
    """
    seed_str = str(seed_song_id)
    
    if seed_str not in stage1_model.wv:
        return []
    
    # most_similar로 유사한 곡 찾기
    similar_songs = stage1_model.wv.most_similar(seed_str, topn=topk+10)
    
    # song_id로 변환하고 seed 제외
    result = []
    for word, score in similar_songs:
        song_id = int(word)
        if song_id != seed_song_id and song_id in IN_CATALOG_SONGS:
            result.append(song_id)
            if len(result) >= topk:
                break
    
    return result


def recommend_stage1_5(seed_song_id: int, topk: int) -> List[int]:
    """
    Stage 1.5: item2vec + artist/genre re-ranking
    TODO: stage1.5.ipynb의 recommend_cf_with_rerank 함수를 import하여 사용
    """
    # 임시: Stage 1과 동일하게 반환 (실제 구현 필요)
    return recommend_stage1(seed_song_id, topk)


def recommend_stage2(seed_song_id: int, topk: int) -> List[int]:
    """
    Stage 2: CF + Audio + Meta 하이브리드
    TODO: Stage 2 구현 후 연결
    """
    raise NotImplementedError("Stage 2 recommend_hybrid를 연결하세요.")


print("✅ 추천 함수 래퍼 정의 완료")
print("  - Stage 1: ✅ 구현됨")
print("  - Stage 1.5: ⚠️ TODO (현재 Stage 1과 동일)")
print("  - Stage 2: ❌ NotImplementedError")

In [ ]:
# ============================================================================
# [Cell 8] 공통 평가 루프 함수 evaluate_model 정의
# ============================================================================

def evaluate_model(
    eval_cases_df: pd.DataFrame,
    recommend_fn: Callable[[int, int], List[int]],
    model_name: str,
    topk: int = 20,
) -> dict:
    """
    공통 평가 함수
    """
    print(f"\n{'='*60}")
    print(f"📊 {model_name} 평가 시작 (TopK={topk})")
    print(f"{'='*60}")
    
    recalls = []
    ndcgs = []
    unique_artists_list = []
    unique_genres_list = []
    max_artist_shares = []
    seed_genre_ratios = []
    
    all_recommended_songs = set()
    
    for idx, row in eval_cases_df.iterrows():
        seed_song_id = row['seed_song_id']
        target_song_ids = row['target_song_ids']
        
        # 추천 생성
        try:
            recommended = recommend_fn(seed_song_id, topk)
        except Exception as e:
            print(f"⚠️ Error at case {idx}: {e}")
            continue
        
        if not recommended:
            continue
        
        # 지표 계산
        recall = recall_at_k(recommended, target_song_ids, topk)
        ndcg = ndcg_at_k(recommended, target_song_ids, topk)
        unique_artists, unique_genres = unique_artists_and_genres_at_k(recommended)
        max_artist_share = max_artist_share_at_k(recommended)
        seed_genre_ratio = seed_genre_ratio_at_k(seed_song_id, recommended)
        
        recalls.append(recall)
        ndcgs.append(ndcg)
        unique_artists_list.append(unique_artists)
        unique_genres_list.append(unique_genres)
        max_artist_shares.append(max_artist_share)
        seed_genre_ratios.append(seed_genre_ratio)
        
        # 카탈로그 커버리지용
        all_recommended_songs.update(recommended)
        
        # 진행상황 출력 (100개마다)
        if (idx + 1) % 100 == 0:
            print(f"  진행: {idx+1:,}/{len(eval_cases_df):,} cases")
    
    # 최종 지표 계산
    metrics = {
        "recall@20": np.mean(recalls),
        "ndcg@20": np.mean(ndcgs),
        "mean_unique_artists@20": np.mean(unique_artists_list),
        "mean_unique_genres@20": np.mean(unique_genres_list),
        "mean_max_artist_share@20": np.mean(max_artist_shares),
        "mean_seed_genre_ratio@20": np.mean(seed_genre_ratios),
        "catalog_coverage@20": len(all_recommended_songs) / len(IN_CATALOG_SONGS),
        "num_cases_evaluated": len(recalls)
    }
    
    # 결과 저장
    metrics_path = os.path.join(EVAL_OUTPUT_DIR, f"metrics_{model_name}.json")
    with open(metrics_path, 'w', encoding='utf-8') as f:
        json.dump(metrics, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ {model_name} 평가 완료")
    print(f"  - 평가된 cases: {metrics['num_cases_evaluated']:,}개")
    print(f"  - Recall@20: {metrics['recall@20']:.4f}")
    print(f"  - nDCG@20: {metrics['ndcg@20']:.4f}")
    print(f"  - 평균 아티스트 수: {metrics['mean_unique_artists@20']:.1f}")
    print(f"  - 평균 장르 수: {metrics['mean_unique_genres@20']:.1f}")
    print(f"  - 저장 경로: {metrics_path}")
    
    return metrics


print("✅ evaluate_model 함수 정의 완료")

In [ ]:
# ============================================================================
# [Cell 9] Stage 1 / 1.5 / 2 평가 실행 및 요약 출력
# ============================================================================

print("\n" + "="*60)
print("🚀 전체 평가 실행 시작")
print("="*60)

# eval_cases 로드
eval_cases_path = os.path.join(EVAL_OUTPUT_DIR, "eval_cases.pkl")
eval_cases_df = pd.read_pickle(eval_cases_path)

# Stage 1 평가
metrics_s1 = evaluate_model(eval_cases_df, recommend_stage1, "stage1", topk=20)

# Stage 1.5 평가
metrics_s1_5 = evaluate_model(eval_cases_df, recommend_stage1_5, "stage1_5", topk=20)

# Stage 2 평가 (구현되어 있으면)
try:
    metrics_s2 = evaluate_model(eval_cases_df, recommend_stage2, "stage2", topk=20)
except NotImplementedError:
    print("\n⚠️ Stage 2는 아직 구현되지 않았습니다.")
    metrics_s2 = None

# 요약 테이블 생성
print("\n" + "="*60)
print("📊 전체 평가 결과 요약")
print("="*60)

summary_data = [
    {"stage": "Stage 1", **metrics_s1},
    {"stage": "Stage 1.5", **metrics_s1_5},
]

if metrics_s2 is not None:
    summary_data.append({"stage": "Stage 2", **metrics_s2})

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# 요약 저장
summary_path = os.path.join(EVAL_OUTPUT_DIR, "evaluation_summary.csv")
summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')

print(f"\n✅ 전체 평가 완료!")
print(f"  - 요약 저장 경로: {summary_path}")
